In [1]:
from src.signal_generator import SignalGenerator
from src.metrics import Metric
from src.data_type import Measurement
import matplotlib.pyplot as plt
from src.data_compressor.other import CompressNTHS
from src.data_compressor.other import CompressMinMax
from src.data_compressor.other import CompressPWP
from src.data_compressor.pip import CompressPIP_ED
from src.data_compressor.pip import CompressPIP_PD
from src.data_compressor.pip import CompressPIP_VD
from src.data_compressor.paa import CompressPAA
from src.data_compressor.paa import CompressPAAVI
from src.data_compressor.paa import CompressByChunk
from src.data_compressor.pla import CompressAPCADFT
from src.data_compressor.pla import CompressAPCAFFT
from src.data_compressor.pla import CompressSTC
from src.data_compressor.pla import CompressHigherDeriveration

In [2]:
from src.data_type import Measurement
from src.data_compressor.compressor import Compressor
from typing import List, Dict
from collections import defaultdict as dict

In [3]:
signal_generator = SignalGenerator(0, 100).with_peaks(3).with_peaks(3, direction=-1).sin(0.2, 0.2)
measurements = [Measurement(measurement, index * 100) for index, measurement in enumerate(signal_generator.data)]

In [51]:
from calendar import c
import math
from re import S
import matplotlib.pyplot as plt
from typing import Tuple, Any, List
import numpy as np
import statistics
from src.data_type import Measurement

class Metric:

  def _strip_data(self, data: List[Measurement]):
    return [item.value for item in data]

  def _interpolate_points(self, x: int, point_A: Measurement, point_B: Measurement) -> float:
    # return ((point_B.value - point_A.value) / (point_B.timestamp - point_A.timestamp)) * x \
    #   + ((point_B.timestamp * point_A.value - point_A.timestamp * point_B.value) / (point_B.timestamp - point_A.timestamp))
    return 5

  def _interpolate_data(self, original: List[Measurement], transformed: List[Measurement]):
    interpolated = []
    current_index = 0
    transformed_count = len(transformed)
    data_original_count = len(original)
    if data_original_count < 2:
      return original
    first = original[0]
    if first.timestamp != transformed[0].timestamp:
      transformed = [Measurement(first.value, first.timestamp)] + transformed

    data_transformed_count = len(transformed)
    last = original[data_original_count - 1]
    if last.timestamp != transformed[data_transformed_count - 1].timestamp:
      transformed.append(Measurement(last.value, last.timestamp))
    
    last_index = 0
    for original_measurement in original:
      for i in range(current_index, transformed_count):
        last_index = i
        print(transformed[i].timestamp, original_measurement.timestamp)
        if transformed[i].timestamp == original_measurement.timestamp:
          print(i)
          interpolated.append(Measurement(original_measurement.value, original_measurement.timestamp))
        elif transformed[i].timestamp > original_measurement.timestamp:
          # print(i)
          current_index = i
          break
      if last_index == current_index:
        point_a = transformed[current_index - 1]
        point_b = transformed[current_index]
        x = original_measurement.timestamp
        y = self._interpolate_points(x, point_a, point_b)
        interpolated.append(Measurement(y, x))
        current_index += 1
    print(len(original))
    print(len(transformed))
    print(len(interpolated))
    return interpolated

  def _prepare_data(self, original: List[Measurement], transformed: List[Measurement]):
    return self._interpolate_data(original, transformed)

  def sum_differences(self, original: List[Measurement], transformed: List[Measurement], use_absolut_value: bool = True) -> float:
    transformed = self._prepare_data(original, transformed)
    differences = [original[i].value - transformed[i].value for i in range(len(original))]
    if use_absolut_value:
      differences = map(math.fabs, differences)
    return sum(differences)


class AlgorythmSelector:

  def __init__(self) -> None:
    self.metrics_containter = Metric()
    self.compressors: Dict[str, Compressor] = {
      'CompressNTHS': CompressNTHS(),
      # 'CompressMinMax': CompressMinMax()
    }

  def set_metrics(self):
    pass # todo
  
  def get_best(self, data: List[Measurement]):
    metric_result = dict()
    for name, compressor in self.compressors.items():
      compressor.set_data(data)
      compressor.compress()
      compressed_data = compressor.compressed_data
      metric_result[name] = {
        # 'compression_rate': self.metrics_containter.compression_ratio(data, compressed_data),
        'sum_differences': self.metrics_containter.sum_differences(data, compressed_data),
      }
    print(metric_result)
    return list(self.compressors.items())[0][1]

AlgorythmSelector().get_best(measurements)

0 0
0
300 0
600 100
900 200
1200 300
1500 400
1800 500
2100 600
2400 700
2700 800
3000 900
3300 1000
3600 1100
3900 1200
4200 1300
4500 1400
4800 1500
5100 1600
5400 1700
5700 1800
6000 1900
6300 2000
6600 2100
6900 2200
7200 2300
7500 2400
7800 2500
8100 2600
8400 2700
8700 2800
9000 2900
9300 3000
9600 3100
9900 3200
9900 3300
100
35
35


IndexError: list index out of range